In [1]:
import os

# Путь к папке с изображениями и аннотациями
images_path = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images'
annotations_path = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/labels'

In [2]:
# Функция для вычисления площади объекта
def calculate_area(annotation):
    _, x_center, y_center, width, height = map(float, annotation.split())
    return width * height

# Проход по файлам в папке аннотаций
for annotation_file in os.listdir(annotations_path):
    if annotation_file.endswith('.txt'):
        annotation_file_path = os.path.join(annotations_path, annotation_file)
        
        # Предположим, что имя изображения хранится в аннотации или следует логике именования
        image_base_name = os.path.splitext(annotation_file)[0]

        # Найдем файл изображения с любым расширением
        image_file_path = None
        for ext in ['.jpg', '.png', '.jpeg', '.bmp']:
            potential_image_path = os.path.join(images_path, image_base_name + ext)
            if os.path.exists(potential_image_path):
                image_file_path = potential_image_path
                break
        
        if image_file_path is None:
            print(f"Изображение для {annotation_file} не найдено.")
            continue

        with open(annotation_file_path, 'r') as file:
            annotations = file.readlines()

        image_area = 1.0  # Площадь изображения в нормированных координатах (1.0 x 1.0)
        for annotation in annotations:
            object_area = calculate_area(annotation)
            if object_area > 0.3 * image_area:
                print(image_file_path)
                break

/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images/images76_jpg.rf.50bb3cee65bd8f2b0198ab4d76d1f36a.jpg
/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images/images150_jpg.rf.f81c7036d8f7b7a66c1bd56d60f5bb7e.jpg
/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images/IMG_20220527_154018_jpg.rf.bb2c5b5736bbc3a8b511539ea2467c58.jpg
/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images/H784200ffa1054790889185cafcfc721aC_jpg.rf.1e1921e1cfdb7103739d56e77495a162.jpg
/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images/image31_png_jpg.rf.fd0534321198f8b9547c006750eb5471.jpg
/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/train/images/a-24-_jpg.rf.f4f30fc6205bed1

In [6]:
import os
import random
import shutil
import cv2

# Путь к папке с изображениями и разметкой
# images_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/valid/images'
# annotations_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/valid/labels'
# output_images_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/valid-clean/images'
# output_annotations_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/fixed wing UAV dataset.v1-fixed-wing-uav.yolov9/valid-clean/labels'

images_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/Helicopter.v1i.yolov9/train/images'
annotations_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/Helicopter.v1i.yolov9/train/labels'
output_images_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/Helicopter.v1i.yolov9/train-clean/images'
output_annotations_dir = '/Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/Helicopter.v1i.yolov9/train-clean/labels'


# Создаем выходные папки, если их нет
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_annotations_dir, exist_ok=True)

def calculate_bbox_area(annotation_file, image_shape):
    h, w = image_shape[:2]
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        total_area = 0
        for line in lines:
            _, x_center, y_center, bbox_width, bbox_height = map(float, line.split())
            bbox_width *= w
            bbox_height *= h
            total_area += bbox_width * bbox_height
    return total_area

# Получаем список всех изображений
images_list = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

# Списки для хранения отфильтрованных изображений
less_than_30_percent = []
more_than_30_percent = []

for image_name in images_list:
    image_path = os.path.join(images_dir, image_name)
    annotation_path = os.path.join(annotations_dir, image_name.replace('.jpg', '.txt'))
    
    # Проверяем наличие файла разметки
    if not os.path.exists(annotation_path):
        continue
    
    # Загружаем изображение и считаем площадь объектов
    image = cv2.imread(image_path)
    image_area = image.shape[0] * image.shape[1]
    bbox_area = calculate_bbox_area(annotation_path, image.shape)
    
    # Вычисляем процент площади, занимаемой объектами
    if bbox_area / image_area < 0.3:
        less_than_30_percent.append(image_name)
    else:
        more_than_30_percent.append(image_name)

# Оставляем случайные 5% из оставшихся изображений
random_5_percent = random.sample(more_than_30_percent, int(len(more_than_30_percent) * 0.05))

# Копируем файлы в выходные папки
for image_name in less_than_30_percent + random_5_percent:
    shutil.copy(os.path.join(images_dir, image_name), output_images_dir)
    shutil.copy(os.path.join(annotations_dir, image_name.replace('.jpg', '.txt')), output_annotations_dir)

print(f"Фильтрация завершена. Новые изображения сохранены в папке: {output_images_dir}")
print(f"Файлы разметки сохранены в папке: {output_annotations_dir}")


Фильтрация завершена. Новые изображения сохранены в папке: /Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/Helicopter.v1i.yolov9/train-clean/images
Файлы разметки сохранены в папке: /Users/nikita/ldt2024/drones/LDT-BPLA-24/datasets/Helicopter.v1i.yolov9/train-clean/labels
